In [1]:
%matplotlib inline
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from foreground_extract_modular import fore_segm

In [2]:
image_files = list('images\\'+i for i in os.listdir('images'))

In [3]:
boxes = pd.read_csv('training.csv')

In [4]:
boxes.head()

,image_name,x1,x2,y1,y2
0,JPEG_20160706_121146_1000145715002.png,115,495,143,325
1,JPEG_20161119_174038_1000690577600.png,23,457,61,409
2,147444927651111470309333776-Roadster-Men-Casua...,37,601,13,470
3,147772332675720161028_161611.png,27,602,162,385
4,1473315333651DeeplearnS11638.png,60,586,174,325


In [5]:
imgnames = []

for i in boxes.values[20:40]:
    img_name = i[0]
    imgnames.append(img_name)

In [6]:
imgnames

['1474638701993DSC06811.png',
 'JPEG_20160628_132442_1000306311172.png',
 '147742527128620161025_151026.png',
 '1480944963239_R2A1990.png',
 'JPEG_20160804_161517_100037187045.png',
 'JPEG_20160709_131920_1000403949668.png',
 '1474711748206DSC07405.png',
 'JPEG_20161124_133940_1000370433134.png',
 'JPEG_20161123_185817_1000936746234.png',
 'JPEG_20160723_192023_100022810186.png',
 'JPEG_20161123_161917_1000691245413.png',
 '147772062874720161028_154604.png',
 '1474634407232DSC05920.png',
 'JPEG_20160810_135200_1000252350484.png',
 'JPEG_20161130_151625_1000513274539.png',
 '1473661629179DeeplearnS11509.png',
 'JPEG_20161117_175829_1000670753549.png',
 'JPEG_20161123_184340_1000982358317.png',
 '147443993217111472450523339-Roadster-Men-Grey-Sneakers-4751472450523145-5.png',
 'JPEG_20161025_131729_1000643972377.png']

In [7]:
box_given_calc = []
for i in imgnames:
    box_given,box_calc = fore_segm(i)
    box_given_calc.append([box_given,box_calc])

In [8]:
box_given_calc

[[[161, 43, 573, 382], [136, 26, 594, 437]],
 [[114, 125, 542, 372], [87, 114, 556, 445]],
 [[42, 119, 479, 364], [27, 29, 497, 384]],
 [[226, 254, 421, 347], [26, 142, 615, 422]],
 [[28, 48, 602, 423], [26, 26, 615, 445]],
 [[24, 19, 477, 472], [26, 26, 500, 445]],
 [[65, 107, 609, 334], [56, 94, 615, 355]],
 [[209, 27, 389, 438], [190, 26, 411, 396]],
 [[175, 164, 486, 299], [40, 72, 507, 445]],
 [[0, 0, 640, 440], [71, 33, 615, 445]],
 [[266, 73, 357, 392], [146, 26, 452, 415]],
 [[86, 92, 566, 338], [34, 26, 587, 437]],
 [[30, 53, 611, 324], [26, 32, 615, 361]],
 [[199, 146, 487, 420], [178, 125, 509, 443]],
 [[34, 70, 601, 382], [26, 46, 615, 401]],
 [[60, 152, 583, 332], [40, 133, 603, 413]],
 [[234, 145, 362, 349], [77, 38, 561, 416]],
 [[112, 169, 483, 337], [92, 26, 604, 361]],
 [[30, 59, 640, 436], [39, 39, 615, 441]],
 [[137, 62, 488, 433], [60, 43, 571, 445]]]

In [9]:
def img_to_numpy(imname):
    img = cv2.imread('images\\'+imname)
    npimg = np.array(img)
    return npimg

In [10]:
npimgs = []
for i in imgnames:
    npimg = img_to_numpy(i)
    npimgs.append(npimg)

In [11]:
npimgs = np.array(npimgs)
npimgs.shape     ## All images

(20, 480, 640, 3)

In [12]:
inputs = []

for i in range(len(box_given_calc)):
    inputs.append([npimgs[i], box_given_calc[i][1]])

npinputs = np.array(inputs)

In [13]:
npinputs.shape   ## Images, Calculated boxes

(20, 2)

In [14]:
calcboxes = []
for i in npinputs:
    calcboxes.append(i[1])

npcalcboxes = np.array(calcboxes)
npcalcboxes.shape

(20, 4)

In [15]:
npoutputs = np.array([i[0] for i in box_given_calc])
npoutputs.shape  ## Given Boxes

(20, 4)

In [16]:
import tensorflow as tf
from tensorflow import keras

In [17]:
box_model = tf.keras.Sequential()

box_model.add(tf.keras.layers.Dense(16, input_dim = 4))
box_model.add(tf.keras.layers.Activation('sigmoid'))

#box_model.add(tf.keras.layers.Dense(8))
#box_model.add(tf.keras.layers.Activation('sigmoid'))

box_model.add(tf.keras.layers.Dense(4, activation = 'linear'))

In [18]:
img_model = tf.keras.Sequential()

img_model.add(keras.layers.Conv2D(16, (3, 3),input_shape = (480, 640, 3), padding = "same", activation = "relu"))

img_model.add(keras.layers.Conv2D(32, (3, 3), padding = "same", activation = "relu"))
img_model.add(keras.layers.MaxPooling2D(pool_size = (2, 2)))

#img_model.add(keras.layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
#img_model.add(keras.layers.MaxPooling2D(pool_size = (2, 2)))

img_model.add(keras.layers.Flatten())
img_model.add(keras.layers.Dense(256, activation = "relu"))
img_model.add(keras.layers.Dense(4, activation = "softmax"))

In [19]:
combinedInput = keras.layers.concatenate([box_model.output, img_model.output])

In [20]:
comb_model = keras.layers.Dense(4, activation="relu")(combinedInput)

In [21]:
final_model = tf.keras.Model(inputs=[box_model.input, img_model.input], outputs=comb_model)

In [22]:
opt = keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200)
final_model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [ ]:
final_model.fit([npcalcboxes,npimgs], npoutputs, epochs = 5, batch_size =2)
final_model.save('final_model.h5')

Epoch 1/5
